In [20]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4 
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)
max_len = 0


def prepare_training(path, n_runs):
    def get_max_len(sequence_list):
        m_len = 0
        for seq in sequence_list:
            if len(seq) > m_len:
                m_len = len(seq)
        max_len = m_len
        print(max_len)
        return m_len
    
    
    def construct_matrix(sequence_list):
        max_len = get_max_len(sequence_list)
        max_len = 500
        train_matrix = np.zeros(shape=(n_runs, max_len, n_features))
        for index,run in enumerate(sequence_list):
            train_matrix[index, :len(run)] = run
        print(train_matrix.shape)
        return train_matrix
        
        
    def stadard_sequences(sequences):
        for i, seq in enumerate(sequences):
            sequences[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
        return sequences       
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        stand_sequences = stadard_sequences(run_list_mix)
        padded_matrix = construct_matrix(stand_sequences)
        return padded_matrix
    
    return read_sequences()
    

train_matrix = prepare_training("Mix_sequences_var_length/run^", n_runs=300) 


414
(300, 500, 4)


In [21]:

from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose
from keras.losses import mse
from keras.models import Model


filters = 50
intermediate_dimension = 30 
latent_dim = 10

def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=2, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def repeat(x):
    steps_matrix = K.ones_like(x[0][:, :, :1])
    latent_matrix = K.expand_dims(x[1], axis=1)
    return K.batch_dot(steps_matrix, latent_matrix)


def create_vae():
    print(max_len)
    inputs = Input(shape=(180, n_features))
    x = inputs
    
    for i in range(2):
        x = Conv1D(filters=filters,
                   kernel_size=2,
                   strides=2,
                   padding='same')(x)
    
    shape = K.int_shape(x)
    
    #before_flattening = LSTM(intermediate_dimension, return_sequences=True)(x)
    x = LSTM(intermediate_dimension)(x)
    #x = Flatten()(x)
    
    
    embeddings = Dense(latent_dim)(x)
    
    z_mean = Dense(latent_dim, name='z_mean',)(embeddings)
    z_log_var = Dense(latent_dim, name='z_log_var')(embeddings)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    #x = Lambda(repeat)([before_flattening, z])
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    
    print(K.int_shape(x))
    for i in range(1):
        x = Conv1DTranspose(input_tensor=x,
                            filters=filters,
                            kernel_size=2,
                            last=False,
                            padding='same')
    
    
    outputs = Conv1DTranspose(input_tensor=x,
                              filters=n_features,
                              kernel_size=2,
                              strides=1,
                              last=True,
                              padding='same')
    
    encoder.summary()
    decoder = Model(latent_inputs, outputs)
    decoder.summary()
    outputs = decoder(encoder.outputs[2])
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    #reconstruction_loss *= sequence_length*4
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), 
                             axis=-1)
    loss = K.mean(reconstruction_loss+0*kl_loss)
    vae = Model(inputs, outputs, name='vae')
    vae.add_loss(loss)
    #vae.summary()
    vae.compile(optimizer='adam')
    return vae, encoder


def create_ae():
    inputs = Input(shape=(500, n_features))
    x = inputs
    for i in range(2):
        x = Conv1D(filters=filters, kernel_size=2, strides=2,
                   activation='relu', padding='same')(x)
    shape = K.int_shape(x)   
    x = LSTM(intermediate_dimension)(x)
    #encoded = Flatten()(x)
    encoded = Dense(latent_dim)(x)
    #x = Lambda(repeat)([before_flattening, encoded])
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x, filters=filters,
                            kernel_size=2, padding='same', last=False)
       
       
    #decoded = LSTM(n_features, return_sequences=True)(x)
    
    output = Dense(n_features, activation='softmax')(x)
    
    encoder = Model(inputs, encoded)
    decoder = Model(latent_inputs, output)
    
    output = decoder(encoder.output)
    sequence_autoencoder = Model(inputs, output)
    sequence_autoencoder.summary()
    sequence_autoencoder.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return sequence_autoencoder, encoder


model, encoder = create_ae()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 500, 4)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 250, 50)           450       
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 125, 50)           5050      
_________________________________________________________________
lstm_4 (LSTM)                (None, 30)                9720      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                310       
_________________________________________________________________
model_11 (Model)             (None, 500, 4)            79054     
Total params: 94,584
Trainable params: 94,584
Non-trainable params: 0
_________________________________________________________________


In [22]:
from keras.callbacks import ModelCheckpoint


def train():
    print(train_matrix.shape)
    model.fit(train_matrix,train_matrix, epochs=250, verbose=1)
    model.save_weights("Models/Weights/VAE_CONV_LSTM_Diff_len_MATRIX_LEN.hdf5")


train()


(300, 500, 4)


Epoch 1/250


 32/300 [==>...........................] - ETA: 1:39 - loss: 1.7788



 64/300 [=====>........................] - ETA: 44s - loss: 1.7587 

...

192/300 [==================>...........] - ETA: 1s - loss: 1.2779

224/300 [=====================>........] - ETA: 0s - loss: 1.2761

256/300 [========================>.....] - ETA: 0s - loss: 1.2795

288/300 [===========================>..] - ETA: 0s - loss: 1.2764

300/300 [==============================] - 3s 10ms/step - loss: 1.2756


In [5]:

model.load_weights("Models/Weights/VAE_CONV_LSTM_Diff_len_MATRIX_LEN.hdf5")


In [23]:

from sklearn.decomposition import PCA

encodings = encoder.predict(train_matrix)
enc_mean, enc_var, z_enc = encodings, encodings, encodings
print(enc_mean.shape, enc_var.shape, z_enc.shape)


def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg


masks = [return_mask(num, np.array(labels))[:, 0] for num in range(0, 9)]


(300, 10) (300, 10) (300, 10)


In [24]:

from mpl_toolkits.mplot3d import Axes3D  


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
    # ax.scatter(unseen_encoding[:,0],
    #            unseen_encoding[:,1],
    #            unseen_encoding[:,2])
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
    # for mask in unseen_mask:
    #     plt.scatter(unseen_encoding[:,0][mask],
    #            unseen_encoding[:,1][mask])
    #     #break
    
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc]
titles = ["MEAN","LOG_VAR","SAMPLED"]
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents.shape)
    print(pca.explained_variance_ratio_)
    plot_pca('Sequences'+titles[i], 0)
    
    principalComponents = enc
    plot_pca('Sequences_Not_Pca'+titles[i], 0)
    


(300, 3)
[0.5058131  0.31088457 0.12133048]


(300, 3)
[0.5058131  0.31088457 0.12133048]


(300, 3)
[0.5058131  0.31088457 0.12133048]


In [25]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

tsne_obj= tsne.fit_transform(enc_input)

print(tsne_obj.shape)

for mask in masks:
    plt.scatter(x=tsne_obj[:, 0][mask], 
                y=tsne_obj[:, 1][mask],
                alpha=0.5)
plt.show()

(300, 2)


In [11]:



unseen_labs = pd.read_csv("Data/Boat_nominal_data/Boat_unseen_labels_mix.csv")
unseen_labs = unseen_labs.drop(columns="Unnamed: 0") 
unseen_labs = np.array(unseen_labs)
unseen_mask = [return_mask(num, np.array(unseen_labs))[:, 0] for num in range(0, 9)]

unseen_sequences_matrix = prepare_training("Mix_sequences_var_length/run_unseen^", 
                                           n_runs=400)
print(unseen_sequences_matrix.shape)
unseen_encoding = encoder.predict(unseen_sequences_matrix)
print(len(unseen_encoding), unseen_encoding[0].shape)

#print(unseen_encoding[0][:, 0])
# def plot_unseen():
#     #CALLS PLOT_PCA with new points
    

900
(400, 900, 4)
(400, 900, 4)
400 (10,)


In [30]:
import ipyvolume as ipv
import numpy as np
x, y, z = unseen_encoding[:,0], unseen_encoding[:,1], unseen_encoding[:,2]

for mask in unseen_mask:
    ipv.scatter(x[mask], y[mask], z[mask], size=0.3, marker="sphere")
ipv.show()